# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy.engine import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import nltk
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import numpy as np


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dahoo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dahoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dahoo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dahoo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('Cleaned_disaster_data',engine)
non_target = ['id_messages', 'original', 'genre', 'message']
X = df.message
Y = df[(df.columns[~df.columns.isin(non_target)])]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokenized = word_tokenize(text)
    
    lemmatizer = WordNetLemmatizer()
    
    tokens = [lemmatizer.lemmatize(w) for w in tokenized if w not in stopwords.words("english")]
    
    tokens = [lemmatizer.lemmatize(w, pos='v') for w in tokens]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:

pipeline = Pipeline([('text_pipeline',Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
                ])),
                    ('MultiOutputClassifier', MultiOutputClassifier(RandomForestClassifier()))])




### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,Y.values, test_size=0.2, shuffle=True)


pipeline.fit(X_train, y_train)

C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('text_pipeline',
                 Pipeline(steps=[('vect',
                                  CountVectorizer(tokenizer=<function tokenize at 0x000001EC40DA7640>)),
                                 ('tfidf', TfidfTransformer())])),
                ('MultiOutputClassifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)

for ind, col in enumerate(Y.columns):
    
    print(col, ':\n', classification_report(y_test[:, ind], y_pred[:, ind]),"\n\n")



related :
               precision    recall  f1-score   support

           0       0.75      0.43      0.55      1286
           1       0.83      0.95      0.89      3920
           2       0.32      0.34      0.33        38

    accuracy                           0.82      5244
   macro avg       0.63      0.57      0.59      5244
weighted avg       0.81      0.82      0.80      5244
 


request :
               precision    recall  f1-score   support

           0       0.91      0.98      0.94      4376
           1       0.83      0.49      0.62       868

    accuracy                           0.90      5244
   macro avg       0.87      0.74      0.78      5244
weighted avg       0.89      0.90      0.89      5244
 


offer :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5244
   macro avg       0.50      0.50    

C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\metr

other_weather :
               precision    recall  f1-score   support

           0       0.95      1.00      0.97      4987
           1       0.50      0.04      0.07       257

    accuracy                           0.95      5244
   macro avg       0.73      0.52      0.52      5244
weighted avg       0.93      0.95      0.93      5244
 


direct_report :
               precision    recall  f1-score   support

           0       0.87      0.98      0.92      4239
           1       0.81      0.36      0.50      1005

    accuracy                           0.86      5244
   macro avg       0.84      0.67      0.71      5244
weighted avg       0.86      0.86      0.84      5244
 




In [7]:
y_test, y_pred

(array([[0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]], dtype=int64),
 array([[0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]], dtype=int64))

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = [{
    'MultiOutputClassifier__estimator__n_estimators' : [10, 100]
}]

cv = GridSearchCV(pipeline, param_grid=parameters)


In [9]:
cv.fit(X_train.values, y_train)

C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\dahoo\anaconda3\lib\site-packages\sk

GridSearchCV(estimator=Pipeline(steps=[('text_pipeline',
                                        Pipeline(steps=[('vect',
                                                         CountVectorizer(tokenizer=<function tokenize at 0x000001EC40DA7640>)),
                                                        ('tfidf',
                                                         TfidfTransformer())])),
                                       ('MultiOutputClassifier',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid=[{'MultiOutputClassifier__estimator__n_estimators': [10,
                                                                             100]}])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_pred_cv = cv.best_estimator_.predict(X_test)

for ind, col in enumerate(Y.columns):
    
    print(col, ':\n', classification_report(y_test[:, ind], y_pred_cv[:, ind]),"\n\n")

related :
               precision    recall  f1-score   support

           0       0.75      0.42      0.54      1286
           1       0.83      0.95      0.89      3920
           2       0.33      0.34      0.34        38

    accuracy                           0.82      5244
   macro avg       0.64      0.57      0.59      5244
weighted avg       0.81      0.82      0.80      5244
 


request :
               precision    recall  f1-score   support

           0       0.91      0.98      0.94      4376
           1       0.84      0.49      0.62       868

    accuracy                           0.90      5244
   macro avg       0.88      0.74      0.78      5244
weighted avg       0.90      0.90      0.89      5244
 


offer :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5244
   macro avg       0.50      0.50    

C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


electricity :
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      5152
           1       1.00      0.05      0.10        92

    accuracy                           0.98      5244
   macro avg       0.99      0.53      0.55      5244
weighted avg       0.98      0.98      0.98      5244
 


tools :
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      5215
           1       0.00      0.00      0.00        29

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      0.99      0.99      5244
 


hospitals :
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      5195
           1       0.00      0.00      0.00        49

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.98      0.

C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dahoo\anaconda3\lib\site-packages\sklearn\metr

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [19]:
import joblib
joblib.dump(cv.best_estimator_, open('models/best_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.